<a href="https://colab.research.google.com/github/HarshaVardhanLanka/springboard_project/blob/main/Day_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install -q -U google-generativeai tavily-python

In [15]:
import os
import google.generativeai as genai
from tavily import TavilyClient
from google.colab import userdata

# 1. Setup API Keys
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY')
except:
    print("Error: Please set your API keys in the Colab Secrets tab (Key icon on the left).")

# 2. Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro-latest') # Updated to gemini-pro-latest

# 3. Configure Tavily
tavily = TavilyClient(api_key=TAVILY_API_KEY)

print("Setup complete! Ready to research.")

Setup complete! Ready to research.


In [16]:
def generate_sub_questions(topic):
    """Step 1: Decompose the topic into sub-questions using Gemini"""
    print(f"🤔 Generating sub-questions for: {topic}...")

    prompt = f"""
    You are a research assistant. Break down the following topic into 3 distinct,
    searchable sub-questions to gather comprehensive information.

    Topic: {topic}

    Output format: Return ONLY the 3 questions separated by newlines.
    Do not number them. Do not add introductory text.
    """

    response = model.generate_content(prompt)
    # Clean up the text to get a list of questions
    questions = [q.strip() for q in response.text.strip().split('\n') if q.strip()]

    return questions[:3] # Ensure we only return 3

def perform_research(questions):
    """Step 2: Search Tavily for each question"""
    print(f"🔎 Searching the web for answers...")
    research_data = []

    for q in questions:
        print(f"   - Searching: {q}")
        # usage of tavily search
        response = tavily.search(query=q, search_depth="basic", max_results=1)

        if response['results']:
            content = response['results'][0]['content']
            research_data.append(f"Question: {q}\nFound Info: {content}")
        else:
            research_data.append(f"Question: {q}\nFound Info: No relevant info found.")

    return "\n\n".join(research_data)

def synthesize_report(topic, research_data):
    """Step 3: Summarize everything into a final paragraph"""
    print(f"📝 Synthesizing final report...")

    prompt = f"""
    You are a professional researcher. Based ONLY on the provided research data below,
    write a concise, high-quality paragraph summarizing the findings regarding the topic: "{topic}".

    Research Data:
    {research_data}

    Final Summary:
    """

    response = model.generate_content(prompt)
    return response.text

In [19]:
# --- Main Workflow ---

# 1. Take User Input
user_topic = input("Enter a research topic (e.g., 'The future of solid state batteries'): ")

if user_topic:
    print("-" * 50)

    # 2. Plan (Generate Questions)
    sub_questions = generate_sub_questions(user_topic)
    print(f"Generated Questions: {sub_questions}")
    print("-" * 50)

    # 3. Act (Search)
    raw_data = perform_research(sub_questions)
    # Optional: Print raw data to see what the agent found
    # print(f"Raw Data:\n{raw_data}")
    print("-" * 50)

    # 4. Synthesize (Final Output)
    final_summary = synthesize_report(user_topic, raw_data)

    print("\n🎯 FINAL RESEARCH SUMMARY:\n")
    print(final_summary)
else:
    print("Please enter a valid topic.")

Enter a research topic (e.g., 'The future of solid state batteries'): Detecting Ocular Toxoplasmosis in eye using XAi
--------------------------------------------------
🤔 Generating sub-questions for: Detecting Ocular Toxoplasmosis in eye using XAi...
Generated Questions: ['What specific XAI techniques and deep learning models are most effective for identifying key biomarkers of ocular toxoplasmosis from retinal fundus images?', "How do explainable AI (XAI) heatmaps and feature attribution methods improve ophthalmologists' diagnostic confidence and accuracy in differentiating ocular toxoplasmosis from other posterior uveitis mimics?", 'What are the primary challenges, including data heterogeneity and model generalizability, in developing and clinically validating XAI systems for the automated diagnosis of active versus scarred ocular toxoplasmosis lesions?']
--------------------------------------------------
🔎 Searching the web for answers...
   - Searching: What specific XAI technique